In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

#generate new axis as required by convolution layer
train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
plt.figure(figsize=(20,20))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(tf.concat([train_images[i],train_images[i],train_images[i]], 2), cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

# Predict using Handwritten image

In [ ]:
file = tf.io.read_file('8.jpg')
img = tf.image.decode_jpeg(file, channels=1)
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.image.resize(img, [28, 28])
#Invert so the Background is Black as in other imgs.
img = (img-1)*-1

plt.imshow(tf.concat([img,img,img], 2))
model.predict(img[tf.newaxis, ...])

## Warum Padding bei Max_Pool?
Pixel am Rand würden sonst evtl. nicht erfasst werden
-> Konnte in den TF-Docs nicht finden, dass die Randpixel gespiegelt werden. Ist bei "MAX" wsl. auch egal ob gespiegelt oder 0

Anmerkung: Padding bei Convolution aus anderen Gründen... Convolution würde auch das Bild verkleinern. Will man nicht immer.. Außerdem würden Merkmale am Rand weniger oft von Filtern erfasst werden.
